In [2]:
import streamlit

In [1]:
import pandas as pd
import sys
sys.path.append("src/")
from statsmodels.tsa.arima_process import ArmaProcess
import numpy as np
import streamlit as st
from utils import myCausalImpact
from utils import plotly_time_series

np.random.seed(12345)



In [6]:


def generate_data() -> pd.DataFrame:
    n = 100
    ar = np.r_[1, 0.9]
    ma = np.array([1])
    arma_process = ArmaProcess(ar, ma)
    x1 = 100 + arma_process.generate_sample(nsample=n)
    x2 = 115 + arma_process.generate_sample(nsample=n)
    y = 1.2 * x1 -0.3*x2 + np.random.normal(size=n)
    t = range(1, n + 1)
    y[70:] += 5

    data = pd.DataFrame({'y': y, 'x1': x1, 'x2': x2, 't': t})

    return data

example_data = generate_data() #type: ignore



In [7]:
vars_to_plot = ['x1', 'y']

time_var = "t"
for plot_var in vars_to_plot:
    fig = plotly_time_series(example_data, time_var, plot_var)


x_vars = [col for col in example_data.columns if col != 'y']
selected_x_vars = x_vars

beg_pre_period = 1
end_pre_period = 50
beg_eval_period = 70
end_eval_period = 99

def estimate_model(df):
    pre_period = [beg_pre_period, end_pre_period]
    eval_period = [beg_eval_period, end_eval_period]
    selected_x_vars_plus_target = ['y'] + selected_x_vars
    ci = myCausalImpact(
        df[selected_x_vars_plus_target], pre_period, eval_period)
    return ci

ci = estimate_model(example_data)

In [8]:
type(ci)

utils.myCausalImpact

In [25]:
def get_n_most_important_vars(trained_c_impact, top_n: int):
    params: pd.Series = trained_c_impact.trained_model.params
    contains_beta = params.index.str.contains("beta")
    does_not_contain_t = params.index != "beta.t"
    params = params[contains_beta & does_not_contain_t]
    params = np.abs(params)
    
    top_n_vars = params.sort_values(ascending=False).index.values[:top_n]
    
    top_n_vars = [var.split(".")[1] for var in top_n_vars]
    return top_n_vars

top_n_vars = get_n_most_important_vars(ci, 1)
top_n_vars

['x1']

In [28]:
type(ci.trained_model)

statsmodels.tsa.statespace.structural.UnobservedComponentsResultsWrapper

In [2]:
from utils import plotly_time_series, estimate_model, get_n_most_important_vars